In [1]:
import datetime as dt
import time
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

import pyxu.abc as pxa
import pyxu.operator as pxop
import pyxu.opt.stop as pxos
#from pyxu.opt.solver.pgd import PGD

import pfw_hawkes as pyfwl
from hawkes_likelihood import HawkesLikelihood
import utils as ut

seed = 1  # None  # for reproducibility
if seed is None:
    seed = np.random.randint(0, 1000)
print(f"Seed: {seed}")

Seed: 1


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

In [3]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
#[kM + k, kM+k + M] pour la k'eme tranche
indices = [index for k in range(3) for index in range(k*3 + k + 1, k*3 + k + 3 + 1)]
print(indices)

print([testarr[(k*3 + k + 1):(k*3 + k + 3 + 1)] for k in range(0, 3)])
print(testarr[indices])

[1, 2, 3, 5, 6, 7, 9, 10, 11]
[array([9, 9, 9]), array([9, 9, 9]), array([9, 9, 9])]
[9 9 9 9 9 9 9 9 9]


In [4]:
# ne pas oublier que le vecteur de parametres theta est dans R^{M^2 + M}
testL1norm = ut.L1NormPartialReg(shape=(1,3*(3+1)), S=indices, regLambda=1)

In [5]:
print(testL1norm(testarr))
print(testL1norm.prox(testarr, tau=1))

81.0
[1. 8. 8. 8. 2. 8. 8. 8. 3. 8. 8. 8.]


In [6]:
supportIndices = [0, 4]
injection = pxop.SubSample(10, supportIndices).T

x = np.ones(len(supportIndices))

print('injection:', injection(x))

injection: [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [7]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
indices = np.array([k + k*3 for k in range(3)], dtype="int32")
print(indices)
print(testarr[indices])

[0 4 8]
[1 2 3]


In [3]:
# Parameter for reconstruction
#lambda_factor = 0.1

## Parameters of the solvers
# Base
remove = True
min_iterations = 1
# Stop
eps = 1e-4
tmax = 15.0
eps_dcv = 1e-2
# PFW
ms_threshold = 0.1  # original value 0.8
init_correction = 1e-1
final_correction = 1e-6
correction_steps = 5

### Generation of the source

In [4]:
path = "simulated_data/events0.csv"
beta = 1
hpL = HawkesLikelihood(path=path, beta=beta)

In [5]:
"""Compute the likelihood on all realizations:
- New likelihood = sum of all likelihoods
= E(A1 * theta) + ... + E(A50 * theta)
Rewrite it as Etilde(Atilde * theta) where 
Atilde = np.vstack([A1, ..., A5-])
and Etilde = somme de E(i'eme tranche pour la i'eme realisation), i=1 -> 50

TODO on the regularization in utils.py
understand why prcoess 7 is zero
understand why FW selects all atoms
"""

"Compute the likelihood on all realizations:\n- New likelihood = sum of all likelihoods\n= E(A1 * theta) + ... + E(A50 * theta)\nRewrite it as Etilde(Atilde * theta) where \nAtilde = np.vstack([A1, ..., A5-])\nand Etilde = somme de E(i'eme tranche pour la i'eme realisation), i=1 -> 50\n\nTODO on the regularization in utils.py\nunderstand why prcoess 7 is zero\nunderstand why FW selects all atoms\n"

In [6]:
hpL.plot_realization()

In [7]:
start = time.time()
lip = hpL.opA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)
#np.linalg.svd to compute the largest singular value which is = to the lipschitz constant for linear operators

Computation time of the Lipschitz constant: 0.0160
Lipschitz constant: 702.2022679988407


In [8]:
# blockDiagA = sp.linalg.block_diag(hpL.A[0], hpL.A[1], hpL.A[2], hpL.A[3], hpL.A[4], hpL.A[5],
#                                   hpL.A[6], hpL.A[7])
# _, s, _ = np.linalg.svd(blockDiagA)
# print(s)
# lip = np.max(s)
# print(blockDiagA.shape)

In [10]:
#lambda_ = lambda_factor * np.linalg.norm(op.T(measurements), np.infty)  # rule of thumb to define lambda
lambda_ = 10e1

### Defining solvers

In [11]:
pfw = pyfwl.PFWLasso(
    hpL,  # replace by likelihood operator hpL
    hpL.opA,  # replace by hpL.opA
    lambda_,
    ms_threshold=ms_threshold,
    init_correction_prec=init_correction,
    final_correction_prec=final_correction,
    remove_positions=remove,
    min_correction_steps=correction_steps,
    show_progress=False,
)

In [12]:
stop_crit = pxos.RelError(
    eps=eps,
    var="objective_func",
    f=None,
    norm=2,
    satisfy_all=True,
)
# alternative stopping criteria
dcv = pyfwl.dcvStoppingCrit(eps_dcv)

# Minimum number of iterations
min_iter = pxos.MaxIter(n=min_iterations)

# Maximum duration
max_duration = pxos.MaxDuration(t=dt.timedelta(seconds=tmax))

stop = (min_iter & stop_crit) | max_duration

# track DCV
track_dcv = pxos.AbsError(eps=1e-10, var="dcv", f=None, norm=2, satisfy_all=True)

### Solving

In [17]:
print("Polyatomic FW: Solving ...")
start = time.time()
pfw.fit(stop_crit= stop | track_dcv, diff_lipschitz=lip**2)
data_p, hist_p = pfw.stats()
time_p = time.time() - start
print("\tSolved in {:.3f} seconds".format(time_p))


Polyatomic FW: Solving ...
diff_lipschitz constant provided.
initial value: -798.7170203317927
new indices [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
new indices [15 21 23 36 37 39 45 47 53 58 60 61 63]
new indices [45 61 63]
new indices [61 63]
new indices [61 63]
new indices [61 63]
new indices [61 63]
new indices [61 63]
new indices [61 63]
new indices [61 63]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new indices [61]
new i

### Evaluation of the solve

In [18]:
print("Final value of dual certificate:\n\tPFW: {:.4f}".format(data_p["dcv"]))
print("Final value of objective function:\n\tPFW : {:.2f}".format(hist_p[-1][-1]))

Final value of dual certificate:
	PFW: 8.2299
Final value of objective function:
	PFW : -31677.56


In [19]:
def supp(arr):
    return np.where(np.abs(arr) > 1e-3)[0]

#print(data_p["x"][supp(data_p["x"])])
#print(data_p["x"])
for i in range(hpL.M):
    print(f"Process {i}:", data_p["x"][(i*(hpL.M + 1)):(i*(hpL.M + 1) + hpL.M + 1)])

Process 0: [2.69067081 0.16970742 1.14315076 2.35685873 0.40063375 2.62848606
 4.13319689 2.08139974 4.25292445]
Process 1: [2.74765118 0.14077397 1.35706726 2.5416206  0.42575375 2.66626552
 4.23478677 2.2319656  4.4792291 ]
Process 2: [2.80927272 0.22389857 1.40386671 2.71668114 0.45315486 2.93636204
 4.39572951 2.3590429  4.59779   ]
Process 3: [2.70835285 0.14742656 1.1657788  2.3885619  0.47182404 2.56826961
 4.20800977 2.1865206  4.27481891]
Process 4: [2.81423099 0.50962774 1.3326676  2.57153477 0.494645   3.25344641
 4.38683201 2.21172654 4.56545559]
Process 5: [2.87242206 0.19235129 1.35221633 2.70367415 0.71841349 2.91207197
 4.92421044 2.48489117 4.71348473]
Process 6: [2.81772858 0.20070606 1.23427669 2.60929258 0.46659112 3.02347045
 4.34701297 2.44619764 4.47547124]
Process 7: [4.30578992 0.         0.         0.         0.         0.
 0.         0.         0.        ]
